# Fine-Tune HyenaDNA On A Downstream Classification Task

In this notebook, we will fine-tune the HynaDNA model on a downstream binary classification task involving cis-regulatory elements.

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/helicalAI/helical/blob/main/examples/notebooks/HyenaDNA-Fine-Tuning.ipynb)

# **What are DNA Features?**
DNA features refer to specific regions or sequences within the DNA molecule that serve distinct biological functions.
For example, promoters, enhancers, and Transcription factors binding sites

These features **regulates** the genetic information and cellular processes.



# **HyenaDNA*** 
Hyena is a genomic foundation model designed to process DNA sequences at single-nucleotide resolution over extremely long contexts—up to 1 million tokens. This represents a 500x increase in context length compared to previous Transformer-based models, which were limited to 512–4,000 tokens due to the quadratic scaling of attention mechanisms.

 Key Innovations
Single-Nucleotide Tokenization: HyenaDNA uses a vocabulary of just five tokens—A, C, G, T, and N—allowing it to model DNA sequences at the finest possible resolution without relying on k-mers or token aggregation 
Hugging Face

Hyena Operator Architecture: Instead of attention, HyenaDNA employs the Hyena operator—a convolution-based mechanism that scales sub-quadratically with sequence length. This enables efficient training and inference on long sequences 
Hugging Face

Sequence Length Warm-Up: To stabilize training on ultra-long sequences, HyenaDNA introduces a scheduling technique that gradually increases sequence length during training. This approach reduces training time by up to 40% and improves accuracy 
ar5iv

In-Context Learning: HyenaDNA demonstrates the first use of in-context learning in genomics, allowing it to adapt to new tasks without updating pretrained model weights. This is achieved through techniques like soft prompting and instruction fine-tuning 


HyenaDNA's capabilities make it suitable for a range of genomic tasks, including:​

Predicting regulatory elements

Analyzing chromatin profiles

Species classification

Identifying enhancers and promoters


### Imports

In [ ]:
from helical.models.hyena_dna import HyenaDNA, HyenaDNAConfig, HyenaDNAFineTuningModel
from datasets import load_dataset
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import torch
import logging, warnings
import numpy as np

logging.getLogger().setLevel(logging.ERROR)

warnings.filterwarnings("ignore")

Make use of GPU if there is a GPU present, otherwise default to the CPU

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

### Dataset Download

- We now use the Hugging Face API to download the [dataset](https://huggingface.co/datasets/InstaDeepAI/nucleotide_transformer_downstream_tasks_revised)
- We select the task from the list of tasks available for this dataset (InstaDepp)

| Task                  | Number of train sequences | Number of test sequences | Number of labels | Sequence length |
| --------------------- | ------------------------- | ------------------------ | ---------------- | --------------- |
| promoter_all          | 30,000                    | 1,584                    | 2                | 300             |
| promoter_tata         | 5,062                     | 212                      | 2                | 300             |
| promoter_no_tata      | 30,000                    | 1,372                    | 2                | 300             |
| enhancers             | 30,000                    | 3,000                    | 2                | 400             |
| enhancers_types       | 30,000                    | 3,000                    | 3                | 400             |
| splice_sites_all      | 30,000                    | 3,000                    | 3                | 600             |
| splice_sites_acceptor | 30,000                    | 3,000                    | 2                | 600             |
| splice_sites_donor    | 30,000                    | 3,000                    | 2                | 600             |
| H2AFZ                 | 30,000                    | 3,000                    | 2                | 1,000           |
| H3K27ac               | 30,000                    | 1,616                    | 2                | 1,000           |
| H3K27me3              | 30,000                    | 3,000                    | 2                | 1,000           |
| H3K36me3              | 30,000                    | 3,000                    | 2                | 1,000           |
| H3K4me1               | 30,000                    | 3,000                    | 2                | 1,000           |
| H3K4me2               | 30,000                    | 2,138                    | 2                | 1,000           |
| H3K4me3               | 30,000                    | 776                      | 2                | 1,000           |
| H3K9ac                | 23,274                    | 1,004                    | 2                | 1,000           |
| H3K9me3               | 27,438                    | 850                      | 2                | 1,000           |
| H4K20me1              | 30,000                    | 2,270                    | 2                | 1,000           |


In [ ]:
label = "promoter_tata"
dataset_train = load_dataset("InstaDeepAI/nucleotide_transformer_downstream_tasks_revised", label, split="train", trust_remote_code=True)
dataset_test = load_dataset("InstaDeepAI/nucleotide_transformer_downstream_tasks_revised", label, split="test", trust_remote_code=True)



In [ ]:
shuffled_dataset = dataset_train.shuffle(seed=42)  # you can set a seed for reproducibility
dataset_train = shuffled_dataset.select(range(100))

unique, counts = np.unique(dataset_train["label"], return_counts=True)
dict(zip(unique, counts))


In [ ]:
shuffled_dataset = dataset_test.shuffle(seed=42)  # you can set a seed for reproducibility
dataset_test = shuffled_dataset.select(range(20))

unique, counts = np.unique(dataset_test["label"], return_counts=True)
dict(zip(unique, counts))

# **HyenaDNA Model Configuration Options**

**model_name**
Specifies the name or variant of the model to use. Examples include:

"hyenadna-tiny-1k-seqlen": a lightweight model for short sequences.

"hyenadna-tiny-1k-seqlen-d256": similar, but with a different embedding dimension.

**batch_size**
Defines how many sequences are processed together during training or inference. Affects memory usage and training speed.



Since this is a binary classification task, only 2 labels are present below

### Define our HyenaDNA fine-tuning model and configs

In [ ]:
hyena_config = HyenaDNAConfig(model_name="hyenadna-tiny-1k-seqlen-d256", batch_size=10, device=device)


hyena_fine_tune = HyenaDNAFineTuningModel(hyena_config, fine_tuning_head="classification", output_size=len(np.unique(dataset_train["label"])))




Fine-Tuning Parameters
hyena_config
Type: HyenaDNAConfig
This is the configuration object for the base HyenaDNA model. It includes all architectural and training parameters used when instantiating the original HyenaDNA model (e.g., number of layers, embedding size, sequence length, dropout rates, etc.). This config ensures consistency between pretraining and fine-tuning.

fine_tuning_head
Type: Literal["classification", "regression"] | HelicalBaseFineTuningHead
Specifies the type of task-specific head to append to the base model for fine-tuning:

"classification": Adds a classification head suitable for categorical outputs (e.g., predicting labels).

"regression": Adds a regression head for continuous-valued outputs.

Alternatively, a custom head can be provided by subclassing HelicalBaseFineTuningHead for advanced use cases.

output_size
Type: Optional[int], default = None
Defines the number of output units in the fine-tuning head:

Required if using a built-in "classification" or "regression" head.

For classification, this typically equals the number of classes.

For regression, this is usually 1 (for scalar output) or more for multi-target regression.

### Process the dataset for both training and testing

In [ ]:
train_dataset = hyena_fine_tune.process_data(dataset_train["sequence"])
test_dataset = hyena_fine_tune.process_data(dataset_test["sequence"])

train_dataset



### Fine-tune the model

For this we set our training and validation datasets along with their labels

In [ ]:
hyena_fine_tune.train(train_dataset=train_dataset, train_labels=dataset_train["label"], validation_dataset=test_dataset, validation_labels=dataset_test["label"], epochs=2, optimizer_params={"lr": 2e-6}, lr_scheduler_params={"name": "linear", "num_warmup_steps": 0})

### Evaluate the fine-tuned model

We now get the outputs of the model on our test dataset and display some results

In [ ]:
outputs = hyena_fine_tune.get_outputs(test_dataset)
print(outputs.shape)

In [ ]:
print(classification_report(dataset_test["label"], outputs.argmax(axis=1)))

In [ ]:
# Compute the confusion matrix
cm = confusion_matrix(dataset_test["label"], outputs.argmax(axis=1))

# Perform row-wise normalization
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Get unique labels in the order they appear in the confusion matrix
unique_labels = np.unique(np.concatenate((dataset_test["label"], outputs.argmax(axis=1))))

# Create and plot the normalized confusion matrix
fig, ax = plt.subplots(figsize=(10, 10))
disp = ConfusionMatrixDisplay(confusion_matrix=cm_normalized, display_labels=unique_labels)
disp.plot(ax=ax, xticks_rotation='vertical', values_format='.2f', cmap='coolwarm')

# Customize the plot
ax.set_title('Normalized Confusion Matrix (Row-wise)')
fig.set_facecolor("none")

plt.tight_layout()
plt.show()